In [25]:
import gzip
import pickle

In [26]:
from wyckoff_transformer.tokenization import load_tensors_and_tokenisers

In [44]:
tensors, tokenizers, engineers = load_tensors_and_tokenisers("mp_20", "with_stop/augmented/harmonic_cluster")

FileNotFoundError: [Errno 2] No such file or directory: '/home/kna/WyckoffTransformer/cache/mp_20/tensors/with_stop/augmented/harmonic_cluster.pkl.gz'

In [46]:
import torch
a={"f": torch.tensor([1,2,3])}

In [48]:
torch.save(a, "test.pt")

In [50]:
torch.load("test.pt", weights_only=True)

{'f': tensor([1, 2, 3])}

In [28]:
tensors['train'].keys()

dict_keys(['wyckoff_letters', 'site_symmetries', 'elements', 'sites_enumeration', 'multiplicity', 'harmonic_site_symmetries', 'spacegroup_number', 'formation_energy_per_atom', 'band_gap', 'harmonic_site_symmetries_augmented', 'pure_sequence_length'])

In [29]:
tensors['train']['harmonic_site_symmetries'].shape

torch.Size([27136, 21, 12])

In [30]:
len(tensors['train']['harmonic_site_symmetries_augmented'])

27136

In [40]:
len(tensors['train']['harmonic_site_symmetries_augmented'][0])

1

In [43]:
len(tensors['train']['harmonic_site_symmetries_augmented'][0][0])

12

In [32]:
with gzip.open("cache/engineers/harmonic_site_symmetries.pkl.gz", "rb") as f:
    data = pickle.load(f)

In [33]:
from preprocess_wychoffs import clasterize_harmonics

In [34]:
distances = clasterize_harmonics(data)

In [35]:
def assign_to_clusters(
    distances: pd.DataFrame):

    remaining_distances = distances.copy().droplevel((0, 1), axis=0)
    assert (remaining_distances.index == np.arange(remaining_distances.shape[0])).all()
    assert (remaining_distances.columns == np.arange(remaining_distances.shape[1])).all()
    mapping = np.empty(distances.shape[0], dtype=int)
    
    while not remaining_distances.empty:
        row, col = np.unravel_index(np.argmin(remaining_distances.values), remaining_distances.shape)
        row_label = remaining_distances.index[row]
        col_label = remaining_distances.columns[col]
        # enum -> cluster
        mapping[row_label] = col_label
        remaining_distances = remaining_distances.drop(row_label, axis=0).drop(col_label, axis=1)
    return pd.Series(mapping, index=distances.index.get_level_values(2))

NameError: name 'pd' is not defined

In [ ]:
mapping = distances.groupby(level=["spacegroup_number", "site_symmetries"]).apply(assign_to_clusters)

In [ ]:
mapping

spacegroup_number  site_symmetries  sites_enumeration
1                  1                0                    5
2                  -1               0                    0
                                    1                    6
                                    2                    3
                                    3                    7
                                                        ..
230                .3.              0                    5
                   .32              0                    5
                   1                0                    5
                   2..              0                    5
                   2.22             0                    5
Length: 1731, dtype: int64

In [ ]:
inverse_index = pd.MultiIndex.from_arrays(
    [mapping.index.get_level_values(0), mapping.index.get_level_values(1), mapping.values],
    names=[mapping.index.names[0], mapping.index.names[1], "cluster-index"]
)

In [ ]:
inverse_mapping = pd.Series(data=mapping.index.get_level_values(2), index=inverse_index)

In [ ]:
inverse_mapping.loc[190]

site_symmetries  cluster-index
-6..             6                0
                 5                1
                 4                2
.2.              3                0
1                0                0
3..              6                0
                 1                1
32.              0                0
m..              3                0
Name: sites_enumeration, dtype: int64

In [ ]:
inverse_mapping.sort_index().loc[190]

site_symmetries  cluster-index
-6..             4                2
                 5                1
                 6                0
.2.              3                0
1                0                0
3..              1                1
                 6                0
32.              0                0
m..              3                0
Name: sites_enumeration, dtype: int64

In [ ]:
mapping.loc[190]

site_symmetries  sites_enumeration
-6..             0                    6
                 1                    5
                 2                    4
.2.              0                    3
1                0                    0
3..              0                    6
                 1                    1
32.              0                    0
m..              0                    3
dtype: int64

In [ ]:
pd.Series(
    data=mapping.index.get_level_values(2),
    index=
)

sites_enumeration
0    1138
1     416
2      92
3      63
4       7
5       7
6       4
7       4
Name: count, dtype: int64

In [ ]:
mapping.value_counts()

0    499
5    257
3    203
2    184
4    179
6    178
1    128
7    103
Name: count, dtype: int64

In [ ]:
mapping.shape

(8,)

In [ ]:
mapping = np.empty(len(d), dtype=int)